# Exemple de notebook - premières analyses des données EDC (Eau Distribuée par Commune)


In [1]:
# Nous commencons par importer les librairies nécessaires pour l'analyse des données.

import duckdb

con = duckdb.connect(database="./../../database/data.duckdb", read_only=True)

In [2]:
# Affichons les tables

con.table("edc_communes").df()

,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07
2,01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07
3,01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07
4,01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07
...,...,...,...,...,...,...
49550,97617,TSINGONI,MIRERENI VILLAGE,976000059,MIRERENI COMBANI,2013-05-30
49551,97617,TSINGONI,MROUALE VILLAGE,976000059,MIRERENI COMBANI,2013-05-30
49552,97617,TSINGONI,TSINGONI CENTRE,976000059,MIRERENI COMBANI,2013-05-30
49553,97701,SAINT-BARTHELEMY,Toute la commune,971000159,SAINT-BARTHELEMY,2010-10-19


In [3]:
con.table("edc_prelevements").df()

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim
0,001,001000003,01007,AMBRONAY,None,None,None,00100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qua...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,N,C
1,001,001000003,01007,AMBRONAY,None,None,None,00100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C
2,001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100139969,2024-03-13,10h46,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C
3,001,001000003,01007,AMBRONAY,None,None,None,00100140209,2024-04-05,07h27,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C
4,001,001000003,01007,AMBRONAY,None,None,None,00100140543,2024-05-13,12h43,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404207,976,976003554,97615,PAMANDZI,976000056,UP PAMANDZI,100 %,97600032142,2024-09-10,09h20,Eau d'alimentation non conforme aux limites de...,MAYOTTE,SMAE,LES EAUX DE MAYOTTE,N,C,C,C
404208,976,976003554,97615,PAMANDZI,None,None,None,97600032321,2024-11-12,10h00,Eau d'alimentation conforme aux exigences de q...,MAYOTTE,SMAE,LES EAUX DE MAYOTTE,C,C,C,C
404209,976,976003883,97601,ACOUA,None,None,None,97600031330,2024-01-11,09h55,Eau d'alimentation conforme aux exigences de q...,MAYOTTE,SMAE,LES EAUX DE MAYOTTE,C,C,C,C
404210,976,976003883,97601,ACOUA,None,None,None,97600031370,2024-01-17,11h00,Eau d'alimentation conforme aux exigences de q...,MAYOTTE,SMAE,LES EAUX DE MAYOTTE,C,C,C,C


In [4]:
con.table("edc_resultats").df().head(20)

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl
0,001,00100138503,ALTMICR,1370.0,ALUMINIUM TOTAL µG/L,Aluminium total µg/l,None,N,L,<10,µg/L,133,None,<=200 µg/L,0.00,7429-90-5,00100147233
1,001,00100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100147232
2,001,00100138503,ASP,6489.0,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.00,None,00100147233
3,001,00100138503,CL2LIB,1398.0,CHLORE LIBRE,Chlore libre,None,N,T,"0,30",mg(Cl2)/L,165,None,None,0.30,None,None
4,001,00100138503,CL2TOT,1399.0,CHLORE TOTAL,Chlore total,None,N,T,"0,34",mg(Cl2)/L,165,None,None,0.34,None,None
5,001,00100138503,CLO2,1754.0,BIOXYDE DE CHLORE MG/L CLO2,Bioxyde de chlore mg/L ClO2,None,N,T,N.M.,mg/L,162,None,None,NaN,10049-04-4,None
6,001,00100138503,COULQ,5900.0,COULEUR (QUALITATIF),Couleur (qualitatif),None,O,T,Aucun changement anormal,SANS OBJET,X,None,None,0.00,None,None
7,001,00100138503,FET,1393.0,FER TOTAL,Fer total,None,N,L,10,µg/L,133,None,<=200 µg/L,10.00,7439-89-6,00100147232
8,001,00100138503,O3,1746.0,OZONE,Ozone,None,N,T,N.M.,mg/L,162,None,None,NaN,10028-15-6,None
9,001,00100138503,ODQ,5901.0,ODEUR (QUALITATIF),Odeur (qualitatif),None,O,L,Aucun changement anormal,SANS OBJET,X,None,None,0.00,None,00100147232


In [5]:
# Chargeons la table edc_communes dans un pandas dataframe, et calculons le nombre de communes

communes = con.table("edc_communes").to_df()
nombre_de_communes = communes.nunique()["inseecommune"]
print(f"nombre_de_communes = {nombre_de_communes}")

nombre_de_communes = 34880


In [9]:
# Faisons une requête SQL en utilisant duckdb via la librarie python pour lister les substances qui ont été recherchées
# et les trier par ordre décroissant de leur nombre d'occurrences

con.sql("""
    SELECT libmajparametre, COUNT(*) as count
    FROM edc_resultats
    GROUP BY libmajparametre
    ORDER BY count DESC
""").show()

┌─────────────────────────────────────┬────────┐
│           libmajparametre           │ count  │
│               varchar               │ int64  │
├─────────────────────────────────────┼────────┤
│ PH                                  │ 338786 │
│ CONDUCTIVITÉ À 25°C                 │ 265188 │
│ TEMPÉRATURE DE L'EAU                │ 258858 │
│ CHLORE LIBRE                        │ 257631 │
│ ESCHERICHIA COLI /100ML - MF        │ 255609 │
│ ENTÉROCOQUES /100ML-MS              │ 255593 │
│ BACTÉRIES COLIFORMES /100ML-MS      │ 255539 │
│ TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU       │ 255307 │
│ CHLORE TOTAL                        │ 254918 │
│ BACT. AÉR. REVIVIFIABLES À 36°-44H  │ 254690 │
│          ·                          │      · │
│          ·                          │      · │
│          ·                          │      · │
│ BENZO(J)FLUORANTHÈNE                │      1 │
│ GIARDIA SPP                         │      1 │
│ CHLOROTHALONIL MÉTABOLITE R611965   │      1 │
│ RÉSIDU SEC À 180° 

In [7]:
# Maintenant configurons l'extension SQL pour Jupyter...

%load_ext sql
%sql duckdb:///../../database/data.duckdb

# ...et faisons la même requête SQL en utilisant l'extension SQL pour Jupyter

%sql SELECT libmajparametre, COUNT(*) as count FROM edc_resultats GROUP BY libmajparametre ORDER BY count DESC;

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

Running query in 'duckdb:///../../database/data.duckdb'

libmajparametre,count
PH,338786
CONDUCTIVITÉ À 25°C,265188
TEMPÉRATURE DE L'EAU,258858
CHLORE LIBRE,257631
ESCHERICHIA COLI /100ML - MF,255609
ENTÉROCOQUES /100ML-MS,255593
BACTÉRIES COLIFORMES /100ML-MS,255539
TURBIDITÉ NÉPHÉLOMÉTRIQUE NFU,255307
CHLORE TOTAL,254918
BACT. AÉR. REVIVIFIABLES À 36°-44H,254690


In [8]:
# Enfin, terminons par lister les prélèvements effectués dans une commune donnée

nomcommune = "TOULOUSE"

con.sql(f"""
    SELECT *
    FROM edc_prelevements
    WHERE nomcommuneprinc = '{nomcommune}'
""").show()

┌─────────┬───────────┬───────────────────┬─────────────────┬───────────────┬───────────────────────────┬───────────────┬───────────────┬────────────┬───────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────┬──────────┬────────────────────┬───────────────────────┬───────────────────────┬────────────────────────────┬────────────────────────────┐
│ cddept  │ cdreseau  │ inseecommuneprinc │ nomcommuneprinc │ cdreseauamont │      nomreseauamont       │ pourcentdebit │ referenceprel │  dateprel  │ heureprel │                                             conclusionprel                                              │       ugelib       │ distrlib │       moalib       │ plvconformitebacterio │ plvconformitechimique │ plvconformitereferencebact │ plvconformitereferencechim │
│ varchar │  varchar  │      varchar      │     varchar     │    varchar    │          varchar          │    varchar    │    varchar    │    dat